In [36]:
import os
import sys
module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
import escalateclient
import importlib
import pandas as pd
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
importlib.reload(escalateclient)
server_url = 'http://localhost:8000'
#server_url = 'https://escalate.cs.haverford.edu/test_server'
username = 'vshekar'
password = 'copperhead123'
client = escalateclient.ESCALATEClient(server_url, username, password)

In [44]:
chem_list = pd.read_csv('Chemicals List.csv')

In [45]:
chem_list = chem_list.fillna('')
chem_list['Material type'] = chem_list['Material type'].str.replace('Gas', 'flux')
chem_list.head()

,BarCode,Chemical Name,Inventory Name,CAS Num,Phase,Purity,Vendor,Material type,molar mass
0,120585,Argon,,7440-37-1,Gas,,CSU Legacy,flux,39.95
1,128224,Benzene,,71-43-2,Liquid,,CSU Legacy,Organic,78.11
2,20790,Boric acid,,10043-35-3,Solid,,CSU Legacy,Organic,61.83
3,207298,Chromium,Powder,7440-47-3,Solid,,CSU Legacy,Inorganic,51.99
4,23852,Cobalt,metal powder,7440-48-4,Solid,,CSU Legacy,Inorganic,58.93


In [46]:
chemical_name = chem_list.iloc[0]['Chemical Name']
material_type_response = client.get_or_create('material-type', data={'description': chem_list.iloc[0]['Material type'].lower()})[0]
material_data = {'description': chemical_name, 'material_type': [material_type_response['url']], 'material_class':'model'}
material_reponse = client.get_or_create('material', data=material_data)


{'description': 'flux'}
GET: OK. Found 1 results
{'description': 'Argon', 'material_type': ['http://localhost:8000/api/material-type/fd3e16a4-3a60-40a6-ad4b-868144c5229d/'], 'material_class': 'model'}
GET: OK. Found 0 results
POST: OK, returning new resource dict


In [47]:
neilson_lab_response = client.get_or_create(endpoint='actor', data={'description': 'Neilson Lab', 'person__isnull': True, 'systemtool__isnull': True})[0]
inventory_data = {
    "description": "Neilson Lab Inventory",
    "owner": neilson_lab_response['url'],
    "lab":neilson_lab_response['url'] 
}
inventory_response = client.get_or_create(endpoint='inventory', data=inventory_data)[0]

{'description': 'Neilson Lab', 'person__isnull': True, 'systemtool__isnull': True}
GET: OK. Found 1 results
{'description': 'Neilson Lab Inventory', 'owner': '73f43230-72f9-4263-adcf-17c38ea29dac', 'lab': '73f43230-72f9-4263-adcf-17c38ea29dac'}
GET: OK. Found 0 results
POST: OK, returning new resource dict


In [48]:
for i, row in chem_list.iterrows():
    chemical_name = row['Chemical Name']
    material_types = row['Material type'].lower().split(',')
    mt_responses = []
    for mt in material_types:
        mt_responses.append(client.get_or_create('material-type', data={'description': mt})[0]) 
    # Add material to db
    material_data = {'description': chemical_name, 'material_type': [mtr['url'] for mtr in mt_responses], 'material_class':'model'}
    material_response = client.get_or_create('material', data=material_data)[0]
    # Add material to inventory
    if row['Inventory Name']:
        description = f"{chemical_name} {row['Inventory Name']}"
    else:
        description = chemical_name
    im_data = {
        "description": description,
        "part_no": f"{row['CAS Num']}",
        "phase": row['Phase'].lower(),
        "inventory": inventory_response['url'],
        "material": material_response['url']
    }
    im_response = client.get_or_create('inventory-material', data=im_data)

{'description': 'flux'}
GET: OK. Found 1 results
{'description': 'Argon', 'material_type': ['http://localhost:8000/api/material-type/fd3e16a4-3a60-40a6-ad4b-868144c5229d/'], 'material_class': 'model'}
GET: OK. Found 0 results
POST: OK, returning new resource dict
{'description': 'Argon', 'part_no': '7440-37-1', 'phase': 'gas', 'inventory': 'e63ff203-834a-44c6-9f9d-8f197d789205', 'material': '5ca56f74-b90c-4b26-8912-e2b4f373c79e'}
GET: OK. Found 0 results
POST: OK, returning new resource dict
{'description': 'organic'}
GET: OK. Found 1 results
{'description': 'Benzene', 'material_type': ['http://localhost:8000/api/material-type/7464cf1a-eda5-411d-815d-572278705752/'], 'material_class': 'model'}
GET: OK. Found 0 results
POST: OK, returning new resource dict
{'description': 'Benzene', 'part_no': '71-43-2', 'phase': 'liquid', 'inventory': 'e63ff203-834a-44c6-9f9d-8f197d789205', 'material': 'a2df28c5-0bab-47d6-9e78-597d5600eaff'}
GET: OK. Found 0 results
POST: OK, returning new resource dic